# import necessary libraries 

In [2]:
import os 
import json 
import pandas as pd 
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI


In [6]:
from dotenv import load_dotenv

load_dotenv()  #take environment variables from .env.

True

In [7]:
KEY=os.getenv("OPENAI_API_KEY")

In [9]:
llm=ChatOpenAI(openai_api_key=KEY,model_name = "gpt-3.5-turbo",temperature=0.3)

In [10]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [38]:
RESPONSE_JSON = {
    "q1":{
        "mcq":"Multiple Choice Question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct answer ",
    },
    "q2":{
        "mcq":"Multiple Choice Question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct answer ",
    },
    "q3":{
        "mcq":"Multiple Choice Question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct answer ",
    },
    "q4":{
        "mcq":"Multiple Choice Question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct answer ",
    },
    "q5":{
        "mcq":"Multiple Choice Question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct answer ",
    },
    
    
    
    
    
}

In [39]:
TEMPLATE = """
Text:{text}
You are an expert MCQ Generator. Generate MCQ based on the given text. is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone}\
The number is {number}. The subject is {subject}. The tone is {tone}. Make sure the question are note repeated and check all the questions to be conforming the text as well
,Make sure to format the response is like RESPONSE_JSON below and use it is as guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}.
"""

In [40]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [41]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [42]:
TEMPLATE2 = """
You are an expert english grammarian and writer,Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the questions and give a complete analysis of the quiz.\
    Only use at most 50 words.\
        if the quiz is not at per the cognitive and analytical abilties of the students,
        updates the quiz quiz questions which needs to changed and change the tone such that it perfectly fits the student abilities 
        Quiz:{quiz}
        
        Check from an expert English Writer of the above quiz:
"""

In [65]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["quiz","subject"],
    template=TEMPLATE
)

In [66]:
evaluation_chain =LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="evaluation",verbose=True)

In [67]:
final_chain = SequentialChain(
    chains=[quiz_chain,evaluation_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","evaluation"],
    verbose=True
)

In [68]:
file_path="/Users/apple/Downloads/MCQGenerator-/data.txt"

In [69]:
with open(file_path,"r") as file:
    TEXT = file.read()

In [70]:
# if you want to serialize the Python dictionary to JSON String 
json_str = json.dumps(RESPONSE_JSON)

In [71]:
NUMBER=5,
SUBJECT="Machine Learning",
TONE ="MEDIUM",

In [72]:
# How to setup tokens usage tracking in Langchain

with get_openai_callback() as cb:
    response = final_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json_str}
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [73]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2880
Prompt Tokens:2074
Completion Tokens:806
Total Cost:0.004723


In [74]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [80]:
print(type(response))

<class 'dict'>


In [82]:
quiz=response.get("quiz")

In [83]:
print(type(quiz))

<class 'str'>


In [84]:
quiz = json.loads(quiz)

In [86]:
#print(type(quiz))
quiz

{'q1': {'mcq': 'What term was coined in 1959 by Arthur Samuel to describe the field of computer gaming and artificial intelligence?',
  'options': {'a': 'Artificial Intelligence',
   'b': 'Machine Learning',
   'c': 'Computer Gaming',
   'd': 'Self-Teaching Computers'},
  'correct': 'b'},
 'q2': {'mcq': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
  'options': {'a': 'Arthur Samuel',
   'b': 'Donald Hebb',
   'c': 'Walter Pitts',
   'd': 'Warren McCulloch'},
  'correct': 'b'},
 'q3': {'mcq': "In the 1960s, what experimental 'learning machine' was developed by Raytheon Company to analyze sonar signals and speech patterns?",
  'options': {'a': 'Cybertron',
   'b': 'HAL 9000',
   'c': 'Deep Blue',
   'd': 'R2-D2'},
  'correct': 'a'},
 'q4': {'mcq': 'According to Tom M. Mitchell, what is the definition of machine learning algorithms?',
  'options': {'a': 'Learning from experience with respect to tasks and performance',
   'b':

In [78]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [79]:
quiz_table_data

[{'MCQ': 'What term was coined in 1959 by Arthur Samuel to describe the field of computer gaming and artificial intelligence?',
  'Choices': 'a: Neural Networks | b: Machine Learning | c: Artificial Intelligence | d: Computer Vision',
  'Correct': 'b: Machine Learning'},
 {'MCQ': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
  'Choices': 'a: Arthur Samuel | b: Walter Pitts | c: Donald Hebb | d: Warren McCulloch',
  'Correct': 'c: Donald Hebb'},
 {'MCQ': "Which company developed the experimental 'learning machine' called Cybertron in the early 1960s?",
  'Choices': 'a: IBM | b: Raytheon Company | c: Microsoft | d: Google',
  'Correct': 'b: Raytheon Company'},
 {'MCQ': 'Who provided a formal definition of the algorithms studied in the machine learning field?',
  'Choices': 'a: Tom M. Mitchell | b: Alan Turing | c: Arthur Samuel | d: Donald Hebb',
  'Correct': 'a: Tom M. Mitchell'},
 {'MCQ': 'What is one of the objectives of 

In [91]:

quiz = pd.DataFrame(quiz_table_data)

In [92]:
quiz

,MCQ,Choices,Correct
0,What term was coined in 1959 by Arthur Samuel ...,a: Neural Networks | b: Machine Learning | c: ...,b: Machine Learning
1,Who published the book 'The Organization of Be...,a: Arthur Samuel | b: Walter Pitts | c: Donald...,c: Donald Hebb
2,Which company developed the experimental 'lear...,a: IBM | b: Raytheon Company | c: Microsoft | ...,b: Raytheon Company
3,Who provided a formal definition of the algori...,a: Tom M. Mitchell | b: Alan Turing | c: Arthu...,a: Tom M. Mitchell
4,What is one of the objectives of modern-day ma...,a: To build robots | b: To predict the weather...,c: To classify data based on models


In [89]:
quiz.to_csv("MachineLearningQuiz.csv",index=False)

In [90]:
from datetime import datetime
datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2024-08-04 12:10:43'